In [8]:
import os
import pandas as pd
import csv
import math
import numpy as np
from ipywidgets import Dropdown

In [9]:
folder_path = 'SaliencyData/Task2'

def load_data(file):
    file_path = os.path.join(folder_path, file)
    data = pd.read_csv(file_path, delimiter=';', usecols=['GazeTime', 'Region', 'Target', 'PosX', 'PosY', 'PosZ', 'PlayerPosX', 'PlayerPosY', 'PlayerPosZ', 'LightIntensity', 'Velocity'])
    data = data[~((data['PosX'] == 0) & (data['PosY'] == 0) & (data['PosZ'] == 0))]

    if len(data) <= 1:  # The data contains only a header
        return None

    # Get the unique regions
    regions = data['Region'].unique()

    # Skip the first region
    for region in regions[1:]:
        # Get the index of the first row of the current region
        first_row_index = data[data['Region'] == region].index[0]
        # Drop the first row of the current region
        data = data.drop(first_row_index)

    # Normalize position data using Min-Max normalization
    pos_columns = ['PosX', 'PosY', 'PosZ', 'PlayerPosX', 'PlayerPosY', 'PlayerPosZ', 'LightIntensity', 'Velocity']
    for column in pos_columns:
        data[column] = (data[column] - data[column].min()) / (data[column].max() - data[column].min())

    return data

# Create a list to store the dataframes
dataframes = [df for df in (load_data(file) for file in os.listdir(folder_path) if file.endswith('.csv')) if df is not None]


In [10]:
def sum(arr):
    ret = 0
    for i in arr:
        ret += i
    return ret

def PercFixInside(dataframe):
    def Compute(region, interest):
        ret = pd.DataFrame()
        target_names = dataframe[dataframe['Region'] == region]['Target'].unique()
        grouped_data = dataframe[dataframe['Region'] == region].groupby('Target')
        gaze = [0] * len(target_names)  # Initialize list with zeros
        perc = [0] * len(target_names)
        ret['Target'] = target_names
        
        for j, target in enumerate(target_names):
            gaze[j] = grouped_data.get_group(target)[interest].sum()
        tot = sum(gaze)

        for i, g in enumerate(gaze):
            perc[i] = (g / tot) * 100 if tot > 0 else 0

        ret['GazeTime'] = gaze
        ret['Percentage'] = perc
        return ret

    regions = dataframe['Region'].unique()

    # Initialize a list to store the DataFrames for each region
    results = []

    for region in regions:
        newdata = Compute(region, 'GazeTime')
        newdata['Region'] = region  # Add a column for the region
        results.append(newdata)

    # Concatenate all the result DataFrames into one
    final_df = pd.concat(results, ignore_index=True)

    return final_df



In [11]:
def NFix(dataframe, threshold, ShowPercentage = False):
    if(threshold < 0):
        threshold = abs(threshold)
    nfix = {}  # Initialize the NFix counter as a dictionary
    counter = {}  # Initialize the counter as a dictionary
    pos = {}  # Variables to track previous position for each target

    for _, row in dataframe.iterrows():
        region = row['Region']
        target = row['Target']
        if region not in counter:
            counter[region] = {}
            nfix[region] = {}
        if target not in counter[region]:
            counter[region][target] = 0
            nfix[region][target] = 0
            pos[region, target] = (None, None)

        counter[region][target] += 1
        pos_x_new, pos_y_new = float(row['PosX']), float(row['PosY'])  # Get the X and Y positions

        # Check if a fixation occurred by comparing the current position with the previous position
        if pos[region, target][0] is not None and pos[region, target][1] is not None:
            pos_diff = abs(pos[region, target][0] - pos_x_new) + abs(pos[region, target][1] - pos_y_new)
            if pos_diff > threshold:
                nfix[region][target] += 1  # Increment the NFix counter for the target

        pos[region, target] = (pos_x_new, pos_y_new)  # Update the previous position for the target

    if ShowPercentage:
        for region in nfix:
            for target in nfix[region]:
                nfix[region][target] = (nfix[region][target] / counter[region][target]) * 100 if counter[region][target] > 0 else 0

    # Convert nested dictionary to DataFrame
    nfix_df = pd.DataFrame([(reg, tar, val) for reg, tar_val in nfix.items() for tar, val in tar_val.items()], 
                           columns=['Region', 'Target', 'NFix'])

    return nfix_df

In [12]:
def ConvergTime(dataframe):
    # Group the data by region and then by target
    dataframe = dataframe[dataframe['Target'] != 'None']
    grouped_data = dataframe.groupby(['Region', 'Target'])

    # Initialize an empty DataFrame to store the results
    results = pd.DataFrame(columns=['Region', 'Target', 'ConvergenceTime'])

    # Iterate over each group (region and target)
    for (region, target), group in grouped_data:
        # Calculate the average gaze time for the current group (target within the region)
        avg_gaze_time = group['GazeTime'].mean()
        
        # Append the result to the results DataFrame
        results = pd.concat([results, pd.DataFrame({'Region': [region], 'Target': [target], 'ConvergenceTime': [avg_gaze_time]})], ignore_index=True)

    return results

In [13]:
def calculate_distance(df):
    # Calculate the Euclidean distance
    df['Distance'] = np.sqrt((df['PosX'] - df['PlayerPosX'])**2 + (df['PosY'] - df['PlayerPosY'])**2 + (df['PosZ'] - df['PlayerPosZ'])**2)
    # Remove rows with 'None' Target
    df = df[df['Target'] != 'None']
    # Group by 'Target' and calculate mean distance
    df = df.groupby('Target')['Distance'].mean().reset_index()
    return df



In [14]:
### gaze_data = dataframes[2]
#PercFixInside(gaze_data)
#NFix(gaze_data, 0.05, False)
#ConvergTime(gaze_data)

In [15]:
# Step 1: Apply PercFixInside to each DataFrame in dataframes and accumulate the results
perc_results = [PercFixInside(df) for df in dataframes]

# Step 2: Concatenate the resulting DataFrames into one
all_perc = pd.concat(perc_results)

# Step 3: Group by 'Region' and 'Target' and calculate the mean
avg_perc = all_perc.groupby(['Region', 'Target']).mean().reset_index()

# This will now be a DataFrame with average 'GazeTime' and 'Percentage' for each 'Target' in each 'Region'
avg_perc

,Region,Target,GazeTime,Percentage
0,0,Cake,0.833128,5.455193
1,0,Donuts,0.783876,5.044166
2,0,HamEgg,0.425360,2.727792
3,0,Hambuger,0.569050,3.781989
4,0,IceCream,0.921574,5.957872
5,0,Milk,0.670057,4.586326
6,0,None,7.765741,52.037075
7,0,Sphere,5.673874,36.473383
8,0,Waffle,0.666699,4.271693


In [16]:
# Step 1: Apply NFix to each DataFrame in dataframes and accumulate the results
nfix_results = [NFix(df, 0.05, False) for df in dataframes]

# Step 2: Concatenate the resulting DataFrames into one
all_nfix = pd.concat(nfix_results)

# Step 3: Group by 'Region' and 'Target' and calculate the mean
avg_nfix = all_nfix.groupby(['Region', 'Target']).mean().reset_index()

# This will now be a DataFrame with average 'Count' and 'Percentage' for each 'Target' in each 'Region'
avg_nfix

,Region,Target,NFix
0,0,Cake,2.473684
1,0,Donuts,2.937500
2,0,HamEgg,2.363636
3,0,Hambuger,2.047619
4,0,IceCream,3.350000
5,0,Milk,2.764706
6,0,None,49.233333
7,0,Sphere,12.541667
8,0,Waffle,2.882353


In [18]:
def overall_average_convergence_time(dataframes):
    # Create a list to store the convergence time dataframes
    convergence_time_dfs = []

    # Loop through each dataframe in dataframes
    for df in dataframes:
        # Calculate the convergence time for each 'Region' and 'Target' in the dataframe
        convergence_time_df = ConvergTime(df)
        # Append the resulting dataframe to the list
        convergence_time_dfs.append(convergence_time_df)

    # Concatenate all convergence time dataframes into a single dataframe
    all_data = pd.concat(convergence_time_dfs)

    # Group by 'Region' and 'Target' and calculate the average convergence time
    all_data = all_data.groupby(['Region', 'Target'])['ConvergenceTime'].mean().reset_index()

    return all_data

# usage
overall_avg_convergence_times = overall_average_convergence_time(dataframes)
overall_avg_convergence_times

,Region,Target,ConvergenceTime
0,0,Cake,0.106622
1,0,Donuts,0.096714
2,0,HamEgg,0.118705
3,0,Hambuger,0.133682
4,0,IceCream,0.125987
5,0,Milk,0.089419
6,0,Sphere,0.209487
7,0,Waffle,0.100399


In [19]:
import numpy as np

def calculate_eccentricity(dataframe):
    # Get vectors from the eye to the object and from the eye straight ahead
    dataframe = dataframe.copy()
    dataframe = dataframe[dataframe['Target'] != 'None']

    eye_to_object = dataframe[['PosX', 'PosY', 'PosZ']].values - dataframe[['PlayerPosX', 'PlayerPosY', 'PlayerPosZ']].values
    straight_ahead = np.array([0, 0, 1])  # This assumes that straight ahead is along the z-axis

    # Normalize the vectors (make them length 1) so that dot product gives the cosine of the angle
    eye_to_object = eye_to_object / np.linalg.norm(eye_to_object, axis=1, keepdims=True)
    straight_ahead = straight_ahead / np.linalg.norm(straight_ahead)

    # Calculate the dot product of the two vectors, which is equal to the cosine of the angle between them
    cos_angle = np.dot(eye_to_object, straight_ahead)

    # Calculate the angle itself (in radians)
    angle = np.arccos(cos_angle)

    # Convert to degrees
    angle = np.degrees(angle)

    # Normalize the angle by the Oculus Quest 2's field of view
    fov = 89
    angle = angle / fov

    # Add the angle to the dataframe
    dataframe['Eccentricity'] = angle

    return dataframe

calculate_eccentricity(dataframes[1])


,GazeTime,Region,Target,PosX,PosY,PosZ,PlayerPosX,PlayerPosY,PlayerPosZ,LightIntensity,Velocity,Distance,Eccentricity
14,0.194981,0,Waffle,0.232151,0.553914,0.877616,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,0.152851,0,Waffle,0.244517,0.611627,0.907053,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,0.111485,0,Waffle,0.262839,0.665166,0.937425,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,0.097593,0,Waffle,0.304192,0.708404,0.977467,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,0.083294,0,Waffle,0.342832,0.741094,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,0.236551,0,Waffle,0.238887,0.752913,0.840223,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,0.069580,0,Waffle,0.234042,0.696965,0.827094,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,0.139156,0,Waffle,0.259420,0.649352,0.816368,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55,0.111273,0,Cake,0.422996,0.363978,0.795576,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56,0.626591,0,Donuts,0.476027,0.369689,0.800745,NaN,NaN,NaN,NaN,NaN,NaN,NaN
